# Harmful Brain Activity Detection

In [1]:
import dask
import os
import pandas as pd
import scipy
from scipy.signal import butter, lfilter
from scipy import signal
from matplotlib import pyplot as plt 
import multiprocessing as mp
import numpy as np
import time
from dask.distributed import Client
import dask.bag as db
from sklearn.model_selection import train_test_split
import shutil
import time
import warnings
from scipy.signal import stft

In [2]:
num_cpus = 24

In [3]:
train_eeg_path = "/scratch/kunigalharish.a/hms-brain-activity-classification/dataset/train_eegs/"
train_csv_path = "/scratch/kunigalharish.a/hms-brain-activity-classification/dataset/train.csv"

In [4]:
df_train = pd.read_csv(train_csv_path)

In [5]:
# create class folders inside dataset folder

for c in df_train['expert_consensus'].unique():
    if not os.path.isdir("dataset_24/"+str(c)):
        os.makedirs("dataset_24/"+str(c))
        

In [6]:
# Function to read eeg and spectrogram data
def read_file(folder):
    files = [os.path.join(folder, file) for file in os.listdir(folder)]
    return files

## Data Pre-Processing

### Noise Removal - Band Pass Filter


In [7]:
## Preprossing improved

def find_eeg(eeg_files_copy, group):
    
    for eeg in eeg_files_copy:
        # get eeg id
        _id = int(eeg.split("/")[-1].split(".")[0])
        
        if _id == group:
            # print(eeg)
            return eeg
        

# Get offsets for a particular eeg subsample from the consolidated eeg
def get_offset(eeg_id):
    """
    Func : Get offsets of a particular consildated eeg to retrieve
    every 50 second subsample from the  consolidated eeg data
    
    params: EEG data id
    
    return: list of offsets for a consolidated eeg dataframe, list of classes
    
    """
    global df_train
    
    _offsets = df_train[(df_train['eeg_id']==eeg_id)]['eeg_label_offset_seconds'].tolist()
    
    _class = df_train[(df_train['eeg_id']==eeg_id)]['expert_consensus'].tolist()
    
    return _offsets, _class

# Get subsample based on offset from eeg data
def Extract_Subsample(eeg_df, offset):
    """
    Extract subsample from the eeg data based on offset
    and return the 50 second long subsample dataframe
    """
    return eeg_df.iloc[offset:offset+50,:-1]
    

In [8]:
# Function to get Bandpass filter coefficients
def get_filter_Coeff(lowcut, highcut, freq, order):
    nyq_freq = 0.5 * freq # Nyquist Frequency
    low = lowcut / nyq_freq
    high = highcut / nyq_freq
    b, a = butter(order, [low, high], btype='band')
    return b,a


# Function to apply the filter 
def Bandpass_filter(data, lowcut, highcut, freq, order=5):
    b,a = get_filter_Coeff(lowcut, highcut, freq, order)
    fil_sig = lfilter(b, a, data)
    return fil_sig


def noise_removal(df_eeg):
    
    # constants for filter function
    lowcut = 0.5
    highcut = 20
    freq = 200

    # filter the eeg sample
    filtered_df = df_eeg.apply(lambda x: Bandpass_filter(x, lowcut, highcut, freq, order=5))
    
    return filtered_df

## Feature Extraction

### EEG to Spectrogram

In [9]:
# Feature Extraction

def normalize_array(arr):
    min_val = np.min(arr)
    max_val = np.max(arr)
    norm_arr = (arr - min_val) / (max_val - min_val) * (255 - 1) + 1
    return norm_arr


def z_score_normalization(data):
    mean_val = np.mean(data)
    std_dev = np.std(data)
    normalized_data = (data - mean_val) / std_dev
    return normalized_data


# Convert EEG to Spectrogram

def EEG_to_Spectrogram(df, path):
    
    # Normalize the values of the spectrogram array to help in generalization
#     spec_df = normalize_array(spec_df)
    norm_df = z_score_normalization(df)
    
    # generate spectrogram from eeg
    freqs, t, spec_df = stft(norm_df, 200, nperseg=50*200)
#     freqs, t, spec_df = signal.spectrogram(norm_df, fs=200)
    
    # Use the absolute value to get the magnitude spectrogram
    spectrogram = np.abs(spec_df)
    
#     spectrogram = np.squeeze(spectrogram, axis=-1).T  

    # Plot the spectrogram using Matplotlib
    plt.imshow(spectrogram, aspect='auto', origin='lower')
    # plt.colorbar(label='Power Spectral Density [dB]')
    plt.axis('off')
    # Save the spectrogram as an image
    plt.savefig(f"{path}.png")

In [10]:

def Feature_Extraction(filtered_df, group, off_class):
    print("Feature Extraction..........................")
    offset, c = off_class
    
    # extract subsamples from consolidated eeg
    df_sub = Extract_Subsample(filtered_df, int(offset))

    # Convert eeg to spectrogram
    path = "dataset_24/"+str(c)+"/"+str(group)+"_"+str(offset)

    EEG_to_Spectrogram(filtered_df, path)
    
# Function to generate spectrograms for each EEG file
def generate_spectrograms_parallel(df_eeg, group, offsets_classes):
    print("generate spectrograms")
    
    # multiprocessing
    global num_cpus

    print(f"No of CPUs : {num_cpus}")
    pool = mp.Pool(processes=num_cpus)
    
    results = pool.starmap(Feature_Extraction, [(df_eeg, group, off_class) for off_class in offsets_classes])
    
    pool.close()
    pool.join()
    
    return True
    

In [11]:
# Main
# EEG data frame
eeg_files = read_file(train_eeg_path)

# Read Spectrogram data
# spec_files = read_file(path_spec)

eeg_files_copy = eeg_files.copy()


In [ ]:

# Main
warnings.filterwarnings('ignore')
start_time = time.time()

group = 0
off_classes = [] 
r = 1
for row in df_train.itertuples():
    
    if group == 0:
        group = row.eeg_id
        off_classes = [(row.eeg_label_offset_seconds, row.expert_consensus)]
    elif row.eeg_id == group:
        off_classes.append((row.eeg_label_offset_seconds, row.expert_consensus))
    elif group != row.eeg_id and group != 0:
        
        # get respective eeg file
        # eeg = find_eeg(eeg_files_copy, group)
        eeg = f"{train_eeg_path}{str(group)}.parquet"
        # print("eeg path ",eeg)
        
        # read eed df
        df_eeg = pd.read_parquet(eeg)
        
        
        # Pre-Processing - Noise Removal
        filtered_df = noise_removal(df_eeg)
        
        # Parallelization of Feature Extraction
        ret = generate_spectrograms_parallel(filtered_df, group, off_classes)
        # eeg_files_copy.remove(eeg)
        group = row.eeg_id 
        off_classes = [(row.eeg_label_offset_seconds, row.expert_consensus)]
    print("r------------- : ", r)
    r+=1
    
elapsed_time = time.time() - start_time

print(f"Elapsed time : {elapsed_time}")

# Write to csv - proc time
with open("proc_time.csv", "a+") as pc:
    pc.write(f'{num_cpus},{elapsed_time}\n')



r------------- :  1
r------------- :  2
r------------- :  3
r------------- :  4
r------------- :  5
r------------- :  6
r------------- :  7
r------------- :  8
r------------- :  9
generate spectrograms
No of CPUs : 24
Feature Extraction..........................
Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................









Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


r------------- :  10
r------------- :  11
generate spectrograms
No of CPUs : 24
Feature Extraction..........................
Feature Extraction..........................


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


r------------- :  12
r------------- :  13
r------------- :  14
r------------- :  15
r------------- :  16
r------------- :  17
r------------- :  18
r------------- :  19
r------------- :  20
r------------- :  21
r------------- :  22
generate spectrograms
No of CPUs : 24
Feature Extraction..........................
Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................











...

Feature Extraction..........................
Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................


Feature Extraction..........................
Feature Extraction..........................Feature Extraction..........................

Feature Extraction..........................Feature Extraction..........................
Feature Extraction..........................
Feature Extraction..........................

Feature Extraction..........................
Feature Extraction..........................
Feature Extraction..........................


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Feature Extraction..........................
Feature Extraction..........................
Feature Extraction..........................Feature Extraction..........................

Feature Extraction..........................
Feature Extraction..........................Feature Extraction..........................Feature Extraction..........................
Feature Extraction..........................Feature Extraction..........................



Feature Extraction..........................
Feature Extraction..........................
Feature Extraction..........................


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

r------------- :  649
r------------- :  650
r------------- :  651
r------------- :  652
r------------- :  653
r------------- :  654
r------------- :  655
r------------- :  656
r------------- :  657
r------------- :  658
r------------- :  659
r------------- :  660
r------------- :  661
r------------- :  662
r------------- :  663
r------------- :  664
r------------- :  665
r------------- :  666
r------------- :  667
r------------- :  668
r------------- :  669
r------------- :  670
r------------- :  671
r------------- :  672
r------------- :  673
r------------- :  674
r------------- :  675
r------------- :  676
r------------- :  677
r------------- :  678
r------------- :  679
r------------- :  680
r------------- :  681
r------------- :  682
r------------- :  683
r------------- :  684
r------------- :  685
r------------- :  686
r------------- :  687
r------------- :  688
r------------- :  689
r------------- :  690
r------------- :  691
r------------- :  692
r------------- :  693
r---------

#### Generating plots for Speed up and Elapsed Time

In [ ]:
df_cpu = pd.read_csv("proc_time.csv")

In [ ]:
# No of CPUs vs Elapsed Time

# Plot Num_CPUs vs Elapsed_time
plt.plot(df_cpu['Num_CPUs'], df_cpu['Elapsed_time'], marker='o', linestyle='-')

# Add labels and title
plt.xlabel('Number of CPUs')
plt.ylabel('Elapsed Time (seconds)')
plt.title('Number of CPUs vs Elapsed Time')

# Show plot
plt.grid(True)

plt.savefig('cpu_elapsed_time_plot.png') 

plt.show()


In [ ]:
# Calculate speedup
df_cpu['Speedup'] = df_cpu['Elapsed_time'][0] / df_cpu['Elapsed_time']

# Plot Num_CPUs vs Elapsed_time
plt.plot(df_cpu['Num_CPUs'], df_cpu['Speedup'], marker='o', linestyle='-')

# Add labels and title
plt.xlabel('Number of CPUs')
plt.ylabel('Speed Up')
plt.title('Number of CPUs vs Speed Up')

# Show plot
plt.grid(True)

plt.savefig('cpu_speedup_plot.png') 

plt.show()
